In [0]:
from __future__ import print_function

import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, TimeDistributed
from tensorflow.keras.layers import LSTM


Using TensorFlow backend.


In [0]:
# Training parameters.
batch_size = 32
num_classes = 10
epochs = 5

In [0]:
# Embedding dimensions.
row_hidden = 128
col_hidden = 128

In [0]:

# The data, split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# Reshapes data to 4D for Hierarchical RNN.

# HRNNs can learn across multiple levels of temporal hierarchy over a complex sequence. Usually, the first recurrent layer of an HRNN encodes a sentence
# (e.g. of word vectors) into a sentence vector. The second recurrent layer then encodes a sequence of such vectors (encoded by the first layer) into a 
# document vector. This document vector is considered to preserve both the word-level and sentence-level structure of the context.
 
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))




In [0]:
# Encodes a row of pixels using TimeDistributed Wrapper.
# TimeDistributed. This wrapper applies a layer to every temporal slice of an input. The input should be at least 3D, and the dimension of index one will be 
# considered to be the temporal dimension. Consider a batch of 32 samples, where each sample is a sequence of 10 vectors of 16 dimensions

encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)




In [0]:
# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))



Epoch 1/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.4523 - accuracy: 0.8491 - val_loss: 0.2613 - val_accuracy: 0.9158
Epoch 2/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.1141 - accuracy: 0.9653 - val_loss: 0.0715 - val_accuracy: 0.9786
Epoch 3/5
1875/1875 [==============================] - 20s 10ms/step - loss: 0.0771 - accuracy: 0.9771 - val_loss: 0.0666 - val_accuracy: 0.9788
Epoch 4/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0630 - accuracy: 0.9817 - val_loss: 0.0472 - val_accuracy: 0.9861
Epoch 5/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0502 - accuracy: 0.9852 - val_loss: 0.0542 - val_accuracy: 0.9825


In [0]:
# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 0.05420488119125366
Test accuracy: 0.9825000166893005
